<a href="https://colab.research.google.com/github/alexyen1006/frgss/blob/master/final_dev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from PIL import Image


# from tensorflow import keras
from tensorflow.keras import layers,models,callbacks
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization


In [ ]:
# Create CNN Model

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"##不要警告
In= Input((60, 200, 3))


out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(In)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)

out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)


out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.4)(out)

out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)

out = Dropout(0.4)(out)
out = [Dense(36, name='digit1', activation='softmax')(out),
       Dense(36, name='digit2', activation='softmax')(out),
       Dense(36, name='digit3', activation='softmax')(out),
       Dense(36, name='digit4', activation='softmax')(out),
       Dense(36, name='digit5', activation='softmax')(out),
       Dense(36, name='digit6', activation='softmax')(out)]

model = Model(inputs=In, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 58, 198, 32)  896         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 56, 196, 32)  9248        conv2d_35[0][0]                  
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 56, 196, 32)  128         conv2d_36[0][0]                  
____________________________________________________________________________________________

In [ ]:
Char_set = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"


def strtoonehot(text):
    labellist = []
    for letter in text:
        onehot = [0 for _ in range(36)]
        num = Char_set.find(letter)
        onehot[num] = 1
        labellist.append(onehot)
    return labellist


In [ ]:
print("Reading training data...")
ans=pd.read_csv('data01_train.csv')
ans=ans.set_index('filename')

start=30500
limit=40500
valimit=5000

for i in range(start,limit):
    if i==start:
        nn=[np.array(Image.open("data01_train/" +str(ans.index[i]) ))/255.0]
    else:
        nn.append(np.array(Image.open("data01_train/" +str(ans.index[i]) ))/255.0)
    if i%1000==0:
        print(i)
train_data=np.stack(nn)

read_label = [strtoonehot(ans.iloc[row][0]) for row in range(start,limit)]

train_label = [[] for _ in range(6)]
for arr in read_label:
    for index in range(6):
        train_label[index].append(arr[index])
train_label = [arr for arr in np.asarray(train_label)]
print("Shape of train data:", train_data.shape)




print("Reading validation data...")

for i in range(limit,limit+valimit):
    if (i-limit)==0:
        vn=[np.array(Image.open("data01_train/" +str(ans.index[i]) ))/255.0]
    else:
        vn.append(np.array(Image.open("data01_train/" +str(ans.index[i]) ))/255.0)
    if (i-limit)%1000==0:
        print(i)
vali_data=np.stack(vn)

read_label = [strtoonehot(ans.iloc[row][0]) for row in range(limit,valimit+limit)]
vali_label = [[] for _ in range(6)]
for arr in read_label:
    for index in range(6):
        vali_label[index].append(arr[index])
vali_label = [arr for arr in np.asarray(vali_label)]
print("Shape of validation data:", vali_data.shape)



Reading training data...
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
Shape of train data: (10000, 60, 200, 3)
Reading validation data...
40500
41500
42500
43500
44500
Shape of validation data: (5000, 60, 200, 3)


In [ ]:

model = models.load_model('cnn_model_3.h5')

In [ ]:
# filepath="/content/drive/My Drive/Colab Notebooks/cnn_model_3.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_digit6_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]


# model = models.load_model('cnn_model_3.h5')
model.fit(train_data, train_label, batch_size=16, epochs=1, verbose=1, validation_data=(vali_data, vali_label))
# model.fit(train_data, train_label, batch_size=16, epochs=40, verbose=1, validation_data=(vali_data, vali_label), callbacks=callbacks_list)
# model.fit(vali_data, vali_label, batch_size=16, epochs=1, verbose=1, validation_data=(train_data, train_label))

# loss, accuracy = model.evaluate(np.array(vali_data), np.array(vali_label), verbose=0)
# print('Test loss:', loss)
# print('Test accuracy:', accuracy)
 
#model.save('/content/drive/My Drive/Colab Notebooks/cnn_model.h5')
print('Test finished')

Train on 10000 samples, validate on 5000 samples
10000/10000 [==============================] - 115s 11ms/sample - loss: 0.2011 - digit1_loss: 0.0327 - digit2_loss: 0.0272 - digit3_loss: 0.0289 - digit4_loss: 0.0332 - digit5_loss: 0.0394 - digit6_loss: 0.0396 - digit1_acc: 0.9903 - digit2_acc: 0.9914 - digit3_acc: 0.9908 - digit4_acc: 0.9893 - digit5_acc: 0.9878 - digit6_acc: 0.9873 - val_loss: 0.2777 - val_digit1_loss: 0.0403 - val_digit2_loss: 0.0344 - val_digit3_loss: 0.0457 - val_digit4_loss: 0.0403 - val_digit5_loss: 0.0599 - val_digit6_loss: 0.0567 - val_digit1_acc: 0.9884 - val_digit2_acc: 0.9888 - val_digit3_acc: 0.9880 - val_digit4_acc: 0.9868 - val_digit5_acc: 0.9826 - val_digit6_acc: 0.9816
Test finished


In [ ]:
model.save('cnn_model_3.h5')

In [ ]:
model = models.load_model('modelcopy/cnn_model_3yy.h5')
loss,b,c,d,e,f,g,h,i,j,k,l,acc = model.evaluate(vali_data, vali_label, verbose=0)

In [ ]:
acc

0.9772

In [ ]:
dev_min=0
dev_range=10000

file_dir='data01_dev/'
csv_dir='data01_dev.csv'
model_name='modelcopy/cnn_model_3just2.h5'


model = models.load_model(model_name)
import csv
dev=pd.read_csv(csv_dir)
dev=dev.set_index('filename')


for i in range(dev_min,dev_range):  
    if i==dev_min:
        nn=[np.array(Image.open(file_dir +str(dev.index[i]) ))/255.0]
    else:
        nn.append(np.array(Image.open(file_dir +str(dev.index[i]) ))/255.0)
    if i%100==0:
        print(i)
    if i==(dev_range-1):
        print('loading finish!')
        

dev_data=np.stack(nn)



prelist=[]

predict = model.predict(dev_data, verbose=0)
print('test finished')
for j in range(0,dev_range-dev_min):  #第幾筆數據
    prestr = ''.join([Char_set[np.argmax(predict[i][j], axis=0)] for i in range(0,6)])#第幾個字
    prelist.append(prestr)
    
print('to str')    



table = [
    ['filename','code']
]
try:
    os.chmod('train.csv',755)
    os.chmod('train.csv',(0o070))
    os.chmod('train.csv',(0o007))
    os.chmod('train.csv',(0o700))
except:
    pass

with open('train.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    if dev_min==0:
        writer.writerows(table)
    for i in range(dev_min,dev_range):

        table=[[dev.index[i],prelist[i-dev_min]]]
        writer.writerows(table)
print('finished')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
loading finish!
test finished
to str
finished
